In [1]:
# rm(list=ls())
require(foreign)
require(readstata13)
require(data.table)
require(MASS)
require(ggplot2)
# require(foreach)
require(gridExtra)
require(stringr)
require(matrixStats)
require(parallel)
require(abind)
require(rlist)
require(assertthat)
require(doParallel)

registerDoParallel(cores = 4)

## Resizing notebook plot space
options(repr.plot.width=16, repr.plot.height=9)

Loading required package: foreign
Loading required package: readstata13
Loading required package: data.table
Loading required package: MASS
Loading required package: ggplot2
Loading required package: gridExtra
Loading required package: stringr
Loading required package: matrixStats
matrixStats v0.51.0 (2016-10-08) successfully loaded. See ?matrixStats for help.
Loading required package: parallel
Loading required package: abind
Loading required package: rlist
Loading required package: assertthat
Loading required package: doParallel
Loading required package: foreach
Loading required package: iterators


## Bring in GDP, GGE, GHES and THE

In [2]:
ghes <- data.table(read.dta13("/home/j/Project/IRH/Forecasting/data_new/stats_archive_new_set/ghes_per_cap_draws_MADcap.dta"))
ghes <- ghes[ , .SD, .SDcols = c("iso3", "year", paste0("draw_",c(1:10000)))]
head(ghes)

iso3,year,draw_1,draw_2,draw_3,draw_4,draw_5,draw_6,draw_7,draw_8,⋯,draw_9991,draw_9992,draw_9993,draw_9994,draw_9995,draw_9996,draw_9997,draw_9998,draw_9999,draw_10000
AFG,1995,2.305364,2.305364,2.305364,2.305364,2.305364,2.305364,2.305364,2.305364,⋯,2.305364,2.305364,2.305364,2.305364,2.305364,2.305364,2.305364,2.305364,2.305364,2.305364
AFG,1996,1.618177,1.618177,1.618177,1.618177,1.618177,1.618177,1.618177,1.618177,⋯,1.618177,1.618177,1.618177,1.618177,1.618177,1.618177,1.618177,1.618177,1.618177,1.618177
AFG,1997,1.566892,1.566892,1.566892,1.566892,1.566892,1.566892,1.566892,1.566892,⋯,1.566892,1.566892,1.566892,1.566892,1.566892,1.566892,1.566892,1.566892,1.566892,1.566892
AFG,1998,1.947385,1.947385,1.947385,1.947385,1.947385,1.947385,1.947385,1.947385,⋯,1.947385,1.947385,1.947385,1.947385,1.947385,1.947385,1.947385,1.947385,1.947385,1.947385
AFG,1999,1.782570,1.782570,1.782570,1.782570,1.782570,1.782570,1.782570,1.782570,⋯,1.782570,1.782570,1.782570,1.782570,1.782570,1.782570,1.782570,1.782570,1.782570,1.782570
AFG,2000,1.485130,1.485130,1.485130,1.485130,1.485130,1.485130,1.485130,1.485130,⋯,1.485130,1.485130,1.485130,1.485130,1.485130,1.485130,1.485130,1.485130,1.485130,1.485130


In [3]:
countries <- unique(ghes[, iso3])

In [4]:
gdp <- data.table(read.dta13('/home/j/Project/IRH/Forecasting/gdp/final/Copula_GDPpc_Draws_REV_linConv_RW_using_capped_MAD_data_chaos_25_10000.dta'))
gdp <- gdp[iso3 %in% countries, .SD, .SDcols = c("iso3", "year", "gdppc_mean", paste0("draw_",c(1:10000)))]
head(gdp)

iso3,year,gdppc_mean,draw_1,draw_2,draw_3,draw_4,draw_5,draw_6,draw_7,⋯,draw_9991,draw_9992,draw_9993,draw_9994,draw_9995,draw_9996,draw_9997,draw_9998,draw_9999,draw_10000
AFG,1979,1188.853,1188.853,1188.853,1188.853,1188.853,1188.853,1188.853,1188.853,⋯,1188.853,1188.853,1188.853,1188.853,1188.853,1188.853,1188.853,1188.853,1188.853,1188.853
AFG,1980,1174.080,1174.080,1174.080,1174.080,1174.080,1174.080,1174.080,1174.080,⋯,1174.080,1174.080,1174.080,1174.080,1174.080,1174.080,1174.080,1174.080,1174.080,1174.080
AFG,1981,1256.343,1256.343,1256.343,1256.343,1256.343,1256.343,1256.343,1256.343,⋯,1256.343,1256.343,1256.343,1256.343,1256.343,1256.343,1256.343,1256.343,1256.343,1256.343
AFG,1982,1340.253,1340.253,1340.253,1340.253,1340.253,1340.253,1340.253,1340.253,⋯,1340.253,1340.253,1340.253,1340.253,1340.253,1340.253,1340.253,1340.253,1340.253,1340.253
AFG,1983,1417.630,1417.630,1417.630,1417.630,1417.630,1417.630,1417.630,1417.630,⋯,1417.630,1417.630,1417.630,1417.630,1417.630,1417.630,1417.630,1417.630,1417.630,1417.630
AFG,1984,1440.854,1440.854,1440.854,1440.854,1440.854,1440.854,1440.854,1440.854,⋯,1440.854,1440.854,1440.854,1440.854,1440.854,1440.854,1440.854,1440.854,1440.854,1440.854


In [5]:
gge <- data.table(read.dta13('/home/j/Project/IRH/GGE/final/Copula_GGE_per_GDP_Draws_REV_linConv_V5_logit_gge_gdp_noConv_MADcaps_FINAL_data_chaos_25_10000.dta'))
gge <- gge[iso3 %in% countries , .SD, .SDcols = c("iso3", "year", "gge_gdp_mean", paste0("draw_",c(1:10000)))]
head(gge)

iso3,year,gge_gdp_mean,draw_1,draw_2,draw_3,draw_4,draw_5,draw_6,draw_7,⋯,draw_9991,draw_9992,draw_9993,draw_9994,draw_9995,draw_9996,draw_9997,draw_9998,draw_9999,draw_10000
AFG,1979,0.08415107,0.08415107,0.08415107,0.08415107,0.08415107,0.08415107,0.08415107,0.08415107,⋯,0.08415107,0.08415107,0.08415107,0.08415107,0.08415107,0.08415107,0.08415107,0.08415107,0.08415107,0.08415107
AFG,1980,0.08316460,0.08316460,0.08316460,0.08316460,0.08316460,0.08316460,0.08316460,0.08316460,⋯,0.08316460,0.08316460,0.08316460,0.08316460,0.08316460,0.08316460,0.08316460,0.08316460,0.08316460,0.08316460
AFG,1981,0.08511585,0.08511585,0.08511585,0.08511585,0.08511585,0.08511585,0.08511585,0.08511585,⋯,0.08511585,0.08511585,0.08511585,0.08511585,0.08511585,0.08511585,0.08511585,0.08511585,0.08511585,0.08511585
AFG,1982,0.08608785,0.08608785,0.08608785,0.08608785,0.08608785,0.08608785,0.08608785,0.08608785,⋯,0.08608785,0.08608785,0.08608785,0.08608785,0.08608785,0.08608785,0.08608785,0.08608785,0.08608785,0.08608785
AFG,1983,0.08645380,0.08645380,0.08645380,0.08645380,0.08645380,0.08645380,0.08645380,0.08645380,⋯,0.08645380,0.08645380,0.08645380,0.08645380,0.08645380,0.08645380,0.08645380,0.08645380,0.08645380,0.08645380
AFG,1984,0.08557231,0.08557231,0.08557231,0.08557231,0.08557231,0.08557231,0.08557231,0.08557231,⋯,0.08557231,0.08557231,0.08557231,0.08557231,0.08557231,0.08557231,0.08557231,0.08557231,0.08557231,0.08557231


In [6]:
THE <- data.table(read.dta13("/home/j/Project/IRH/Forecasting/data_new/stats_archive_new_set/the_per_cap_draws_MADcap.dta"))
THE <- THE[ , .SD, .SDcols = c("iso3", "year", paste0("draw_",c(1:10000)))]
head(THE)

iso3,year,draw_1,draw_2,draw_3,draw_4,draw_5,draw_6,draw_7,draw_8,⋯,draw_9991,draw_9992,draw_9993,draw_9994,draw_9995,draw_9996,draw_9997,draw_9998,draw_9999,draw_10000
AFG,1995,51.24436,51.24436,51.24436,51.24436,51.24436,51.24436,51.24436,51.24436,⋯,51.24436,51.24436,51.24436,51.24436,51.24436,51.24436,51.24436,51.24436,51.24436,51.24436
AFG,1996,49.13425,49.13425,49.13425,49.13425,49.13425,49.13425,49.13425,49.13425,⋯,49.13425,49.13425,49.13425,49.13425,49.13425,49.13425,49.13425,49.13425,49.13425,49.13425
AFG,1997,48.47850,48.47850,48.47850,48.47850,48.47850,48.47850,48.47850,48.47850,⋯,48.47850,48.47850,48.47850,48.47850,48.47850,48.47850,48.47850,48.47850,48.47850,48.47850
AFG,1998,48.84705,48.84705,48.84705,48.84705,48.84705,48.84705,48.84705,48.84705,⋯,48.84705,48.84705,48.84705,48.84705,48.84705,48.84705,48.84705,48.84705,48.84705,48.84705
AFG,1999,45.25033,45.25033,45.25033,45.25033,45.25033,45.25033,45.25033,45.25033,⋯,45.25033,45.25033,45.25033,45.25033,45.25033,45.25033,45.25033,45.25033,45.25033,45.25033
AFG,2000,46.57129,46.57129,46.57129,46.57129,46.57129,46.57129,46.57129,46.57129,⋯,46.57129,46.57129,46.57129,46.57129,46.57129,46.57129,46.57129,46.57129,46.57129,46.57129


### The first step is to compute the correlation of in-sample data. Let's subset those out first

In [7]:
gdp_insample <- gdp[year<=2014 & year>=1995 , .SD, .SDcols = c("iso3", "year", "gdppc_mean")]
colnames(gdp_insample) <- c("iso3", "year", "mean.gdp")
gge_insample <- gge[year<=2014 & year>=1995 , .SD, .SDcols = c("iso3", "year", "gge_gdp_mean")]
colnames(gge_insample) <- c("iso3", "year", "mean.gge")
ghes_insample <- ghes[year<=2014 & year>=1995 , .SD, .SDcols = c("iso3", "year", "draw_1")]
colnames(ghes_insample) <- c("iso3", "year", "mean.ghes")
THE_insample <- THE[year<=2014 & year>=1995 , .SD, .SDcols = c("iso3", "year", "draw_1")]
colnames(THE_insample) <- c("iso3", "year", "mean.THE")

### Merge and compute correlation

In [8]:
insample_data_merged <- merge(x=gdp_insample, y=gge_insample, by=c("iso3", "year"), all.y=T)
insample_data_merged <- merge(x=insample_data_merged, y=ghes_insample, by=c("iso3", "year"), all.y=T)
insample_data_merged <- merge(x=insample_data_merged, y=THE_insample, by=c("iso3", "year"), all.y=T)
head(insample_data_merged)

iso3,year,mean.gdp,mean.gge,mean.ghes,mean.THE
AFG,1995,780.7482,0.08294755,2.305364,51.24436
AFG,1996,757.2020,0.08125266,1.618177,49.13425
AFG,1997,738.9581,0.08178695,1.566892,48.47850
AFG,1998,725.0324,0.08308152,1.947385,48.84705
AFG,1999,709.2279,0.08222921,1.782570,45.25033
AFG,2000,677.5062,0.08024725,1.485130,46.57129


### Compute the correlation

In [9]:
print(corr_mat<-cor(insample_data_merged[, .( mean.gdp, mean.gge, mean.ghes, mean.THE)], method = "spearman"))

           mean.gdp  mean.gge mean.ghes  mean.THE
mean.gdp  1.0000000 0.4131485 0.9500292 0.9565547
mean.gge  0.4131485 1.0000000 0.5501293 0.4911001
mean.ghes 0.9500292 0.5501293 1.0000000 0.9744619
mean.THE  0.9565547 0.4911001 0.9744619 1.0000000


### Neal's multivariate copula function

In [10]:
draw2Dcopula <- function(X, cor_mat, df_return = F){
  L <- dim(X)[2]
  D <- dim(X)[3]
  Xsum <- apply(X, c(2, 3), sum)
  mvdat <- mvrnorm(n=D, mu=0 * 1:L, Sigma=cor_mat, empirical=TRUE)
  ranks <- apply(mvdat, 2, rank, ties.method="first")
  sortedXsim <- apply(Xsum, 1, function(x) sort(x, index.return=TRUE)$ix)
  sortedX <- X
  for(i in 1:L){
    sortedX[,i,] <- X[,i,sortedXsim[,i]]
  }
  Xcorr <- sortedX
  for(i in 1:L){
    Xcorr[,i,] <- sortedX[,i,ranks[,i]]
  }
  if (df_return==T) {
    return(data.frame(reshape2::melt(Xcorr)))
    }
      else {
          Xcorr
      }
}

### Merge the draws of LDI and Education, reshape super long and convert to arrays

In [11]:
gdp_fc <- gdp[, .SD, .SDcols = c("iso3","year", paste0("draw_", c(1:10000)))]
gdp_fc <- gdp_fc[, variable := "GDPpc"]
gdp_fc <- setcolorder(gdp_fc, c("iso3","year", "variable", paste0("draw_", c(1:10000))) )
gdp_fc <- melt(gdp_fc, id.vars=c("iso3","year", "variable"), variable.name = "draw_num", value.name = "data")
head(gdp_fc)

iso3,year,variable,draw_num,data
AFG,1979,GDPpc,draw_1,1188.853
AFG,1980,GDPpc,draw_1,1174.080
AFG,1981,GDPpc,draw_1,1256.343
AFG,1982,GDPpc,draw_1,1340.253
AFG,1983,GDPpc,draw_1,1417.630
AFG,1984,GDPpc,draw_1,1440.854


In [12]:
gge_fc <- gge[, .SD, .SDcols = c("iso3","year", paste0("draw_", c(1:10000)))]
gge_fc <- gge_fc[, variable := "GGE_GDP"]
gge_fc <- setcolorder(gge_fc, c("iso3","year", "variable", paste0("draw_", c(1:10000))) )
gge_fc<- melt(gge_fc, id.vars=c("iso3","year", "variable"), variable.name = "draw_num", value.name = "data")
head(gge_fc)

iso3,year,variable,draw_num,data
AFG,1979,GGE_GDP,draw_1,0.08415107
AFG,1980,GGE_GDP,draw_1,0.08316460
AFG,1981,GGE_GDP,draw_1,0.08511585
AFG,1982,GGE_GDP,draw_1,0.08608785
AFG,1983,GGE_GDP,draw_1,0.08645380
AFG,1984,GGE_GDP,draw_1,0.08557231


In [13]:
ghes_fc <- ghes[, .SD, .SDcols = c("iso3","year", paste0("draw_", c(1:10000)))]
ghes_fc <- ghes_fc[, variable := "GHES_pc"]
ghes_fc <- setcolorder(ghes_fc, c("iso3","year", "variable", paste0("draw_", c(1:10000))) )
ghes_fc<- melt(ghes_fc, id.vars=c("iso3","year", "variable"), variable.name = "draw_num", value.name = "data")
head(ghes_fc)

iso3,year,variable,draw_num,data
AFG,1995,GHES_pc,draw_1,2.305364
AFG,1996,GHES_pc,draw_1,1.618177
AFG,1997,GHES_pc,draw_1,1.566892
AFG,1998,GHES_pc,draw_1,1.947385
AFG,1999,GHES_pc,draw_1,1.782570
AFG,2000,GHES_pc,draw_1,1.485130


In [14]:
THE_fc <- THE[, .SD, .SDcols = c("iso3","year", paste0("draw_", c(1:10000)))]
THE_fc <- THE_fc[, variable := "THE_pc"]
THE_fc <- setcolorder(THE_fc, c("iso3","year", "variable", paste0("draw_", c(1:10000))) )
THE_fc<- melt(THE_fc, id.vars=c("iso3","year", "variable"), variable.name = "draw_num", value.name = "data")
head(THE_fc)

iso3,year,variable,draw_num,data
AFG,1995,THE_pc,draw_1,51.24436
AFG,1996,THE_pc,draw_1,49.13425
AFG,1997,THE_pc,draw_1,48.47850
AFG,1998,THE_pc,draw_1,48.84705
AFG,1999,THE_pc,draw_1,45.25033
AFG,2000,THE_pc,draw_1,46.57129


In [15]:
system.time(all_data_binded <- rbind(gdp_fc, gge_fc, ghes_fc, THE_fc))
all_data_binded <- all_data_binded[year >=1995,]
tail(all_data_binded)

   user  system elapsed 
 50.024 187.702 249.614 

iso3,year,variable,draw_num,data
ZMB,2035,THE_pc,draw_10000,362.0940
ZMB,2036,THE_pc,draw_10000,410.0595
ZMB,2037,THE_pc,draw_10000,400.4243
ZMB,2038,THE_pc,draw_10000,423.8123
ZMB,2039,THE_pc,draw_10000,414.0031
ZMB,2040,THE_pc,draw_10000,419.2039


In [16]:
# system.time(myarray2<- mclapply(countries, 
#                                 function(x) reshape2::acast(all_data_binded[iso3 == x ,], 
#                                                             iso3 ~  year ~ variable ~ draw_num, 
#                                                       value.var = "data"), mc.cores=6, mc.preschedule = F))



system.time(
    myarray2 <- foreach(i = countries) %dopar% {
       temp <- reshape2::acast(all_data_binded[iso3 == paste0(i) ,], 
              iso3 ~  year ~ variable ~ draw_num,  value.var = "data")
    return(temp)
        }
)

myarray2 <- abind(myarray2, along=1)
str(myarray2)

   user  system elapsed 
257.467 223.770 165.906 

 num [1:184, 1:46, 1:4, 1:10000] 781 2728 4044 54756 95020 ...
 - attr(*, "dimnames")=List of 4
  ..$ : chr [1:184] "AFG" "AGO" "ALB" "AND" ...
  ..$ : chr [1:46] "1995" "1996" "1997" "1998" ...
  ..$ : chr [1:4] "GDPpc" "GGE_GDP" "GHES_pc" "THE_pc"
  ..$ : chr [1:10000] "draw_1" "draw_2" "draw_3" "draw_4" ...


### Copula'in time

In [18]:
system.time(Xcorr_array <- mclapply(countries, 
                                     function(x) cbind(x, draw2Dcopula(myarray2[paste0(x),,,], 
                                     corr_mat , df_return = T)), mc.cores=6, mc.preschedule = F ))



system.time(test2 <- list.stack(Xcorr_array, data.table=T))
colnames(test2) <- c("iso3", "year", "variable", "draw_num", "data")
str(test2)
head(test2)
tail(test2)                  

   user  system elapsed 
255.820 272.496 110.223 

   user  system elapsed 
 79.154  45.955 126.381 

Classes ‘data.table’ and 'data.frame':	338560000 obs. of  5 variables:
 $ iso3    : Factor w/ 184 levels "AFG","AGO","ALB",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ year    : int  1995 1996 1997 1998 1999 2000 2001 2002 2003 2004 ...
 $ variable: Factor w/ 4 levels "GDPpc","GGE_GDP",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ draw_num: Factor w/ 10000 levels "draw_1","draw_2",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ data    : num  781 757 739 725 709 ...
 - attr(*, ".internal.selfref")=<externalptr> 


iso3,year,variable,draw_num,data
AFG,1995,GDPpc,draw_1,780.7482
AFG,1996,GDPpc,draw_1,757.2020
AFG,1997,GDPpc,draw_1,738.9581
AFG,1998,GDPpc,draw_1,725.0324
AFG,1999,GDPpc,draw_1,709.2279
AFG,2000,GDPpc,draw_1,677.5062


iso3,year,variable,draw_num,data
ZMB,2035,THE_pc,draw_10000,319.3948
ZMB,2036,THE_pc,draw_10000,312.5184
ZMB,2037,THE_pc,draw_10000,330.2646
ZMB,2038,THE_pc,draw_10000,340.7149
ZMB,2039,THE_pc,draw_10000,327.9692
ZMB,2040,THE_pc,draw_10000,332.1186


In [19]:
### Stack into a DT
test3 <- dcast(test2, iso3 + year + variable ~ draw_num, value.var = c("data"))
head(test3)

iso3,year,variable,draw_1,draw_2,draw_3,draw_4,draw_5,draw_6,draw_7,⋯,draw_9991,draw_9992,draw_9993,draw_9994,draw_9995,draw_9996,draw_9997,draw_9998,draw_9999,draw_10000
AFG,1995,GDPpc,780.74816512,780.74816512,780.74816512,780.74816512,780.74816512,780.74816512,780.74816512,⋯,780.74816512,780.74816512,780.74816512,780.74816512,780.74816512,780.74816512,780.74816512,780.74816512,780.74816512,780.74816512
AFG,1995,GGE_GDP,0.08294755,0.08294755,0.08294755,0.08294755,0.08294755,0.08294755,0.08294755,⋯,0.08294755,0.08294755,0.08294755,0.08294755,0.08294755,0.08294755,0.08294755,0.08294755,0.08294755,0.08294755
AFG,1995,GHES_pc,2.30536438,2.30536438,2.30536438,2.30536438,2.30536438,2.30536438,2.30536438,⋯,2.30536438,2.30536438,2.30536438,2.30536438,2.30536438,2.30536438,2.30536438,2.30536438,2.30536438,2.30536438
AFG,1995,THE_pc,51.24436235,51.24436235,51.24436235,51.24436235,51.24436235,51.24436235,51.24436235,⋯,51.24436235,51.24436235,51.24436235,51.24436235,51.24436235,51.24436235,51.24436235,51.24436235,51.24436235,51.24436235
AFG,1996,GDPpc,757.20201261,757.20201261,757.20201261,757.20201261,757.20201261,757.20201261,757.20201261,⋯,757.20201261,757.20201261,757.20201261,757.20201261,757.20201261,757.20201261,757.20201261,757.20201261,757.20201261,757.20201261
AFG,1996,GGE_GDP,0.08125266,0.08125266,0.08125266,0.08125266,0.08125266,0.08125266,0.08125266,⋯,0.08125266,0.08125266,0.08125266,0.08125266,0.08125266,0.08125266,0.08125266,0.08125266,0.08125266,0.08125266


In [20]:
### Validation
test4 <- melt(test3, id.vars=c("iso3", "year", "variable"), value.name = "data", variable.name = "draw_num")
test4 <- dcast(test4, iso3 + year + draw_num ~ variable)
test4 <- test4[,  GGE_pc := GGE_GDP * GDPpc]
head(test4)

Using 'data' as value column. Use 'value.var' to override


iso3,year,draw_num,GDPpc,GGE_GDP,GHES_pc,THE_pc,GGE_pc
AFG,1995,draw_1,780.7482,0.08294755,2.305364,51.24436,64.76115
AFG,1995,draw_2,780.7482,0.08294755,2.305364,51.24436,64.76115
AFG,1995,draw_3,780.7482,0.08294755,2.305364,51.24436,64.76115
AFG,1995,draw_4,780.7482,0.08294755,2.305364,51.24436,64.76115
AFG,1995,draw_5,780.7482,0.08294755,2.305364,51.24436,64.76115
AFG,1995,draw_6,780.7482,0.08294755,2.305364,51.24436,64.76115


In [21]:
test4 <- test4[, GDP_minus_GHES := GDPpc - GHES_pc]
test4 <- test4[, GDP_minus_GGE := GDPpc - GGE_pc]
test4 <- test4[, GGE_minus_GHES := GGE_pc - GHES_pc]
test4 <- test4[, GDP_minus_THE := GDPpc - THE_pc]
head(test4)

iso3,year,draw_num,GDPpc,GGE_GDP,GHES_pc,THE_pc,GGE_pc,GDP_minus_GHES,GDP_minus_GGE,GGE_minus_GHES,GDP_minus_THE
AFG,1995,draw_1,780.7482,0.08294755,2.305364,51.24436,64.76115,778.4428,715.987,62.45578,729.5038
AFG,1995,draw_2,780.7482,0.08294755,2.305364,51.24436,64.76115,778.4428,715.987,62.45578,729.5038
AFG,1995,draw_3,780.7482,0.08294755,2.305364,51.24436,64.76115,778.4428,715.987,62.45578,729.5038
AFG,1995,draw_4,780.7482,0.08294755,2.305364,51.24436,64.76115,778.4428,715.987,62.45578,729.5038
AFG,1995,draw_5,780.7482,0.08294755,2.305364,51.24436,64.76115,778.4428,715.987,62.45578,729.5038
AFG,1995,draw_6,780.7482,0.08294755,2.305364,51.24436,64.76115,778.4428,715.987,62.45578,729.5038


In [22]:
## Subset out where inequalities break
test5 <- test4[GDP_minus_GHES < 0 | GDP_minus_GGE < 0 | GGE_minus_GHES < 0 | GDP_minus_THE < 0,]
bads <- unique(test5[,draw_num])

In [23]:
length(bads)

[1] 2014

In [24]:
## Drop the bads
test4 <- test4[!draw_num %in% bads,]

In [ ]:
## Test!!
summary(test4[, GDP_minus_GHES])
summary(test4[, GDP_minus_GGE])
summary(test4[, GGE_minus_GHES])
summary(test4[, GDP_minus_THE])

    Min.  1st Qu.   Median     Mean  3rd Qu.     Max. 
   194.4   3330.0  10390.0  17810.0  25820.0 408100.0 

     Min.   1st Qu.    Median      Mean   3rd Qu.      Max. 
     0.08   2553.00   7695.00  13350.00  19260.00 286500.00 

    Min.  1st Qu.   Median     Mean  3rd Qu.     Max. 
     0.1    713.4   2384.0   4453.0   6623.0 160700.0 

In [ ]:
## Let's melt and cast out and rename and sample 1000
test6 <- melt(test4[, .(iso3, year, draw_num, GDPpc, GGE_GDP, GHES_pc, GGE_pc, THE_pc)], id.vars = c("iso3", "year", "draw_num"), value.name = "data")
test6 <- dcast(test6, iso3 + year + variable ~ draw_num, value.var = "data")
dim(test6)
head(test6)

In [ ]:
## Rename
len_draw <- length(grep("draw_", names(test6), value = TRUE))
colnames(test6) <- c("iso3", "year", "variable", paste0("draw_",c(1:len_draw)))
# head(test6)
test7 <- test6[, .SD , .SDcols = c("iso3", "year", "variable", paste0("draw_", sample(len_draw, 1000,replace = F)))]
colnames(test7) <- c("iso3", "year", "variable", paste0("draw_",c(1:1000)))
head(test7)

In [ ]:
GDP_sorted <- test7[variable=="GDPpc", .SD, .SDcols = c("iso3","year", paste0("draw_", c(1:1000)))]
colnames(GDP_sorted) <- c("iso3","year", paste0("draw_", c(1:1000)))
GGE_sorted <- test7[variable=="GGE_GDP", .SD, .SDcols = c("iso3","year", paste0("draw_", c(1:1000)))]
colnames(GGE_sorted) <- c("iso3","year", paste0("draw_", c(1:1000)))
GHES_sorted <- test7[variable=="GHES_pc", .SD, .SDcols = c("iso3","year", paste0("draw_", c(1:1000)))]
colnames(GHES_sorted) <- c("iso3","year", paste0("draw_", c(1:1000)))
THE_sorted <- test7[variable=="THE_pc", .SD, .SDcols = c("iso3","year", paste0("draw_", c(1:1000)))]
colnames(THE_sorted) <- c("iso3","year", paste0("draw_", c(1:1000)))

In [ ]:
head(GDP_sorted)
head(GGE_sorted)
head(GHES_sorted)
head(THE_sorted)

In [ ]:
## Save out
fwrite(GDP_sorted, "/home/j/Project/IRH/Forecasting/data_new/GDPpc_sorted.csv" )
fwrite(GGE_sorted, "/home/j/Project/IRH/Forecasting/data_new/GGE_GDP_sorted.csv" )
fwrite(GHES_sorted, "/home/j/Project/IRH/Forecasting/data_new/GHES_pc_sorted.csv" )
fwrite(THE_sorted, "/home/j/Project/IRH/Forecasting/data_new/THE_pc_sorted.csv" )

fwrite(GDP_sorted[year==2014 | year == 2040, ], "/home/j/Project/IRH/Forecasting/data_new/GDPpc_sorted_2014_2040.csv" )
fwrite(GGE_sorted[year==2014 | year == 2040, ], "/home/j/Project/IRH/Forecasting/data_new/GGE_GDP_sorted_2014_2040.csv" )
fwrite(GHES_sorted[year==2014 | year == 2040, ], "/home/j/Project/IRH/Forecasting/data_new/GHES_pc_sorted_2014_2040.csv" )
fwrite(THE_sorted[year==2014 | year == 2040, ], "/home/j/Project/IRH/Forecasting/data_new/THE_pc_sorted_2014_2040.csv" )